# Data Loading

In [0]:
!pip install tqdm

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-ccf9d2f0-4d5a-48f3-baf3-fe11b92afb4a/bin/python -m pip install --upgrade pip' command.


In [0]:
import data_cleaning
from importlib import reload
reload(data_cleaning)


Out[11]: <module 'data_cleaning' from '/Workspace/Users/cjault@uw.edu/data_cleaning.py'>

In [0]:
artwork, artist, medium = data_cleaning.load_data()

  0%|          | 0/477804 [00:00<?, ?it/s]

  0%|          | 0/477804 [00:00<?, ?it/s]

In [0]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, ArrayType

from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Read Online CSV") \
    .getOrCreate()

In [0]:
artwork_schema = StructType([
    StructField("Object ID", IntegerType(), nullable=False),
    StructField("Object Name", StringType(), nullable=True),
    StructField("Is Highlight", BooleanType(), nullable=True),
    StructField("Country", StringType(), nullable=True),
    StructField("Period", StringType(), nullable=True),
    StructField("Culture", StringType(), nullable=True),
    StructField("Gallery Number", IntegerType(), nullable=True),
    StructField("Department", StringType(), nullable=True),
    StructField("Medium IDs", ArrayType(IntegerType()), nullable=True),
    StructField("Artist IDs", ArrayType(IntegerType()), nullable=True)
])

artist_schema = StructType([
    StructField("ID", IntegerType(), nullable=False),
    StructField("Name", StringType(), nullable=True),
    StructField("Gender", StringType(), nullable=True)
])
medium_schema = StructType([
    StructField("ID", IntegerType(), nullable=False),
    StructField("Material", StringType(), nullable=True)
])

In [0]:
artwork = artwork[['Object ID', 'Object Name', 'Is Highlight', 'Country', 'Period', 'Culture', 'Gallery Number', 'Department', 'Medium IDs', 'Artist IDs']]
medium = medium[['ID', 'Material']]
artist = artist[['ID', 'Name', 'Gender']]

# deal with missing values so pyspark can read them
n_artwork = artwork.copy()
n_artwork['Gallery Number'] = artwork['Gallery Number'].fillna(np.nan).replace([np.nan], [None])
n_artwork['Artist IDs'] = artwork['Artist IDs'].fillna(np.nan).replace([np.nan],[None])
n_artwork['Medium IDs'] = artwork['Medium IDs'].fillna(np.nan).replace([np.nan],[None])


In [0]:
# Create PySpark Tables
df_artwork = spark.createDataFrame(data=n_artwork, schema=artwork_schema)
df_artist = spark.createDataFrame(data=artist, schema=artist_schema)
df_medium = spark.createDataFrame(data=medium, schema=medium_schema)

df_artwork.createOrReplaceTempView("artwork")
df_medium.createOrReplaceTempView("medium")
df_artist.createOrReplaceTempView('artist')

## Query 1
Are certain types of artworks highlighted more than others? Specifically, are artworks in certain galleries/departments highlighted more than others?

In [0]:
query1 = """
SELECT  Department, `Gallery Number`, COUNT(*) AS HighlightedCount
FROM artwork
WHERE `Is Highlight` = True
GROUP BY Department, `Gallery Number`
ORDER BY HighlightedCount DESC
"""

result1 = spark.sql(query1)
result1.show()

+--------------------+--------------+----------------+
|          Department|Gallery Number|HighlightedCount|
+--------------------+--------------+----------------+
|       The Libraries|          null|             347|
|   Costume Institute|          null|             134|
|   The American Wing|          null|             127|
|Modern and Contem...|          null|             125|
|         Photographs|          null|             112|
|Arts of Africa, O...|          null|              92|
| Drawings and Prints|          null|              87|
|         Islamic Art|          null|              52|
| Musical Instruments|           684|              52|
|Robert Lehman Col...|          null|              41|
|           Asian Art|          null|              31|
|   The American Wing|           706|              27|
|       The Libraries|          1004|              27|
| Musical Instruments|          null|              26|
|   The American Wing|           746|              25|
| Musical 

In [0]:
query1a = """
SELECT  Department, COUNT(*) AS HighlightedCount
FROM artwork
WHERE `Is Highlight` = True
GROUP BY Department
ORDER BY HighlightedCount DESC
"""

result1a = spark.sql(query1a)
result1a.show()

+--------------------+----------------+
|          Department|HighlightedCount|
+--------------------+----------------+
|   The American Wing|             434|
|       The Libraries|             376|
|Modern and Contem...|             179|
|   Costume Institute|             134|
|  European Paintings|             125|
|         Islamic Art|             120|
| Greek and Roman Art|             115|
|        Egyptian Art|             114|
|         Photographs|             113|
| Musical Instruments|             105|
|Arts of Africa, O...|             104|
|Robert Lehman Col...|              98|
| Drawings and Prints|              89|
|European Sculptur...|              82|
|Ancient Near East...|              75|
|       The Cloisters|              58|
|           Asian Art|              55|
|      Arms and Armor|              54|
|        Medieval Art|              54|
+--------------------+----------------+



In [0]:
query1b = """
SELECT  `Gallery Number`, COUNT(*) AS HighlightedCount
FROM artwork
WHERE `Is Highlight` = True AND `Gallery Number` IS NOT NULL
GROUP BY `Gallery Number`
ORDER BY HighlightedCount DESC
"""

result1b = spark.sql(query1b)
result1b.show()

+--------------+----------------+
|Gallery Number|HighlightedCount|
+--------------+----------------+
|           684|              53|
|           706|              27|
|          1004|              27|
|           746|              25|
|           681|              22|
|           403|              22|
|           304|              19|
|           743|              15|
|            14|              15|
|           119|              14|
|           704|              13|
|           762|              12|
|           153|              12|
|           136|              12|
|           405|              12|
|           700|              11|
|           162|              11|
|           964|              11|
|           121|              11|
|           373|              11|
+--------------+----------------+
only showing top 20 rows



## Question 2
Whose artwork has been acquired most by the museum? What about by period, gender, culture, medium, department, and country?

In [0]:
from pyspark.sql.functions import explode

# Register the exploded DataFrame as a SQL temporary view
exploded_df = df_artwork.select("Object ID", "Period", 'Department', 'Culture', 'Country', explode("Artist IDs").alias("ArtistID"))

exploded_df.createOrReplaceTempView("artwork_exp")

print('\n Artist')
query1 = """
    SELECT 
        artist.Name, 
        COUNT(artwork_exp.`Object ID`) AS Count 
    FROM 
        artwork_exp 
    JOIN 
        artist ON artist.ID = artwork_exp.ArtistID 
    GROUP BY 
        artist.Name 
    ORDER BY 
        COUNT(artwork_exp.`Object ID`) DESC 
    LIMIT 10
"""
result = spark.sql(query1)
result.show()

print('\n Gender')
query2 = """
    SELECT 
        artist.Gender, 
        COUNT(artwork_exp.`Object ID`) AS Count 
    FROM 
        artwork_exp 
    JOIN 
        artist ON artist.ID = artwork_exp.ArtistID 
    GROUP BY 
        artist.Gender 
    ORDER BY 
        COUNT(artwork_exp.`Object ID`) DESC 
    LIMIT 10
"""
result = spark.sql(query2)
result.show()

print('\n Period')
query3 = """
    SELECT 
        artwork.Period, 
        COUNT(artwork.`Object ID`) AS Count 
    FROM 
        artwork 
    WHERE 
        artwork.Period IS NOT NULL 
    GROUP BY 
        artwork.Period 
    ORDER BY 
        COUNT(artwork.`Object ID`) DESC 
    LIMIT 10
"""
result = spark.sql(query3)
result.show()

print('\n Department')
query4 = """
    SELECT 
        artwork.Department, 
        COUNT(artwork.`Object ID`) AS Count 
    FROM 
        artwork 
    GROUP BY 
        artwork.Department 
    ORDER BY 
        COUNT(artwork.`Object ID`) DESC 
    LIMIT 10
"""
result = spark.sql(query4)
result.show()

print('\n Culture')
query5 = """
    SELECT 
        artwork.Culture, 
        COUNT(artwork.`Object ID`) AS Count 
    FROM 
        artwork 
    WHERE 
        artwork.Culture IS NOT NULL 
    GROUP BY 
        artwork.Culture 
    ORDER BY 
        COUNT(artwork.`Object ID`) DESC 
    LIMIT 10
"""
result = spark.sql(query5)
result.show()

print('\n Country')
query6 = """
    SELECT 
        artwork.Country, 
        COUNT(artwork.`Object ID`) AS Count 
    FROM 
        artwork 
    WHERE 
        artwork.Country IS NOT NULL 
    GROUP BY 
        artwork.Country 
    ORDER BY 
        COUNT(artwork.`Object ID`) DESC 
    LIMIT 10
"""

result = spark.sql(query6)

result.show()


print('\n Medium')

exploded_df = df_artwork.select("Object ID", explode("Medium IDs").alias("MediumID"))

exploded_df.createOrReplaceTempView("artwork_exp")

query7 = """
SELECT 
    medium.material as Material, count(`Object ID`) AS Count 
FROM 
    artwork_exp 
JOIN 
    medium ON medium.ID  = artwork_exp.`MediumID` 
GROUP BY 
    medium.material 
ORDER BY 
    count(`Object ID`) DESC LIMIT 10"""
result = spark.sql(query7)
result.show()


 Artist
+--------------------+-----+
|                Name|Count|
+--------------------+-----+
|        Walker Evans| 7668|
| W. Duke, Sons & Co.| 5615|
|             Unknown| 4479|
|Kinney Brothers T...| 4449|
|      Allen & Ginter| 4314|
|American Tobacco ...| 3275|
|      Brewster & Co.| 3146|
|   Thomas Rowlandson| 2969|
|   Goodwin & Company| 2778|
|      Jacques Callot| 2366|
+--------------------+-----+


 Gender
+------+------+
|Gender| Count|
+------+------+
|  Male|404164|
|Female| 14556|
+------+------+


 Period
+--------------------+-----+
|              Period|Count|
+--------------------+-----+
|         New Kingdom|12043|
|          Edo period| 9125|
|             Archaic| 8166|
|           Classical| 6979|
|        Qing dynasty| 6491|
|      Middle Kingdom| 4687|
|   Archaic/Classical| 2166|
|         Late Period| 1820|
|      Early Imperial| 1523|
|Third Intermediat...| 1357|
+--------------------+-----+


 Department
+--------------------+------+
|          Departme

## Question 3
What is the distribution of the number of artists per artwork?

In [0]:
from pyspark.sql.functions import size

query = """
    SELECT 
        CASE WHEN size(artwork.`Artist IDs`) = -1 THEN 'unknown' ELSE CAST(size(artwork.`Artist IDs`) AS STRING) END as TeamSize,
        CAST(COUNT(artwork.`Object ID`) / (SELECT COUNT(*) FROM artwork) AS DECIMAL(10, 10)) as Ratio
    FROM 
        artwork
    GROUP BY 
        size(artwork.`Artist IDs`)
    ORDER BY 
        size(artwork.`Artist IDs`)
"""
result = spark.sql(query)

result.show()

+--------+------------+
|TeamSize|       Ratio|
+--------+------------+
| unknown|0.4254589748|
|       1|0.3906727445|
|       2|0.1185360524|
|       3|0.0443445430|
|       4|0.0121723552|
|       5|0.0038593231|
|       6|0.0014461997|
|       7|0.0014252706|
|       8|0.0005734569|
|       9|0.0002532419|
|      10|0.0002260341|
|      11|0.0001276674|
|      12|0.0001381320|
|      13|0.0001067383|
|      14|0.0000732518|
|      15|0.0000753447|
|      16|0.0000523227|
|      17|0.0000334865|
|      18|0.0000586014|
|      19|0.0000355794|
+--------+------------+
only showing top 20 rows



## Question 4
Are there any artworks with the same title?

In [0]:
query = """
SELECT 
    artwork.`Object Name`, COUNT(artwork.`Object ID`) as Count 
FROM 
    artwork  
GROUP BY  
    artwork.`Object Name` 
ORDER BY 
    COUNT(artwork.`Object ID`) 
DESC LIMIT 10
"""
result = spark.sql(query)

result.show()

+--------------+-----+
|   Object Name|Count|
+--------------+-----+
|         Print|99439|
|    Photograph|28458|
|       Drawing|25788|
|          Book|13393|
|      Fragment| 9566|
|Kylix fragment| 8927|
|         Piece| 8630|
|      Painting| 5932|
|      Negative| 5928|
|          Bowl| 3617|
+--------------+-----+



## Query 5
Of the ten countries from which art is most frequently acquired, which artists are most common?

In [0]:
# Query 5 needs Query 1 to run so putting it here 

query1 = """
SELECT Country, COUNT(*) AS ArtworkCount
FROM artwork
WHERE Country IS NOT NULL
GROUP BY Country
ORDER BY ArtworkCount DESC
"""

result1 = spark.sql(query1)
result1.show()

+--------------------+------------+
|             Country|ArtworkCount|
+--------------------+------------+
|               Egypt|       31446|
|       United States|        9590|
|                Iran|        6595|
|                Peru|        3427|
|              France|        1966|
|     Byzantine Egypt|        1673|
|              Mexico|        1563|
|               India|        1416|
|           Indonesia|        1391|
|             England|        1256|
|               China|         938|
|              Turkey|         921|
|             Germany|         900|
|    Papua New Guinea|         881|
|             Nigeria|         647|
|               Italy|         546|
|               Syria|         528|
|Democratic Republ...|         507|
|               Spain|         432|
|                Iraq|         427|
+--------------------+------------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import explode
df_artwork_exploded = df_artwork.withColumn("Artist IDs", explode(df_artwork['Artist IDs']))

result1.createOrReplaceTempView("result1")
df_artwork_exploded.createOrReplaceTempView("artwork_exploded")

query5 = """
SELECT artist.Name as ArtistName, TopArtists.ArtworkCount FROM 
(SELECT `Artist IDs` as `Artist ID`, COUNT(*) as ArtworkCount FROM artwork_exploded
WHERE Country IN (SELECT Country FROM result1 ORDER BY ArtworkCount DESC LIMIT 5)
GROUP BY `Artist IDs`
ORDER BY ArtworkCount DESC
LIMIT 10) as TopArtists, artist
WHERE TopArtists.`Artist ID` = artist.ID
ORDER BY TopArtists.ArtworkCount DESC
"""

result5 = spark.sql(query5)
result5.show()

+--------------------+------------+
|          ArtistName|ArtworkCount|
+--------------------+------------+
|Louis Comfort Tif...|         621|
|     Tiffany Studios|         537|
|Tiffany Glass and...|         364|
|Tiffany Glass Com...|         199|
|Union Porcelain W...|         184|
|Abu'l Qasim Firdausi|         162|
|Nina de Garis Davies|         155|
|Boston & Sandwich...|         147|
|      Walter Tyndale|         134|
|       Tiffany & Co.|         131|
+--------------------+------------+



## Query 6
Are artworks from specific time periods more likely to be highlighted (displayed) than others?

In [0]:
query6 = """
SELECT Period, 
  (SUM(CASE WHEN `Is Highlight` = TRUE THEN 1 ELSE 0 END) / COUNT(*)) * 100  AS PercentHighlighted
FROM artwork
WHERE Period IS NOT NULL
GROUP BY Period
ORDER BY PercentHighlighted DESC
"""

result6 = spark.sql(query6)
result6.show()

+--------------------+------------------+
|              Period|PercentHighlighted|
+--------------------+------------------+
|      Timurid period|             100.0|
|Late Early Cyclad...|             100.0|
|  Parthian or Kushan|             100.0|
|     Final Neolithic|             100.0|
|      Pandyan period|             100.0|
|      Solanki period|             100.0|
|Shang dynasty–Wes...|             100.0|
|Third Intermediat...|             100.0|
|Early Tokugawa pe...|             100.0|
|            Tokugawa|             100.0|
|Late Period or Ea...|             100.0|
|early Ptolemaic P...|             100.0|
|  Hellenistic period|              50.0|
|  Late Helladic IIIC|              50.0|
|early Eastern Jav...|              50.0|
|late Anuradhapura...|              50.0|
|        Ming Dynasty|              40.0|
|Early Cycladic II...| 33.33333333333333|
|             Severan| 28.57142857142857|
|            Augustan|              25.0|
+--------------------+------------

## Logistic Regression
Predict the possibility of being highlighted.

In [0]:
from pyspark.ml import Transformer
from pyspark.ml.param import Param, Params
from pyspark.ml.param.shared import HasInputCol, HasOutputCol
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, IntegerType

class MultiHotEncoder(Transformer, HasInputCol, HasOutputCol, DefaultParamsReadable, DefaultParamsWritable):
    def __init__(self, inputCol=None, outputCol=None):
        super(MultiHotEncoder, self).__init__()
        self._setDefault(inputCol=None, outputCol=None)
        self._set(inputCol=inputCol, outputCol=outputCol)

    def setInputCol(self, value):
        return self._set(inputCol=value)

    def getInputCol(self):
        return self.getOrDefault(self.inputCol)

    def setOutputCol(self, value):
        return self._set(outputCol=value)

    def getOutputCol(self):
        return self.getOrDefault(self.outputCol)

    def _transform(self, dataset):
        input_col = self.getInputCol()
        output_col = self.getOutputCol()

        def multi_hot_encode(arr):
            unique_values = set(arr)
            return [1 if val in unique_values else 0 for val in range(max(unique_values) + 1)]

        multi_hot_encode_udf = udf(multi_hot_encode, ArrayType(IntegerType()))

        return dataset.withColumn(output_col, multi_hot_encode_udf(input_col))


In [0]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
# Create a StringIndexer for each categorical column
artwork_df = df_artwork.withColumn("Is Highlight_numeric", col("Is Highlight").cast("integer"))
artwork_df = artwork_df.dropna()
indexers = [
    StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid="keep")
    for col in ["Country", "Period", "Culture", "Department"]
]

# Create a OneHotEncoder for each indexed categorical column
encoders = [
    OneHotEncoder(inputCol=col+"_index", outputCol=col+"_encoded")
    for col in ["Country", "Period", "Culture", "Department"]
]

# Create a MultiHotEncoder for the MediumIDs column
# medium_encoder = MultiHotEncoder(inputCol="Medium IDs", outputCol="Medium IDs_encoded")
 
# Create a MultiHotEncoder for the ArtistIDs column
# artist_encoder = MultiHotEncoder(inputCol="Artist IDs", outputCol="Artist IDs_encoded")

# Create a VectorAssembler to combine all features into a single vector column
assembler = VectorAssembler(
    inputCols=["Gallery Number"
    # , "Medium IDs_encoded", "Artist IDs_encoded"
    ] + [col+"_encoded" for col in ["Country", "Period", "Culture", "Department"]],
    outputCol="features",
    handleInvalid="skip"
)

# Create a LogisticRegression classifier
logistic_regression = LogisticRegression(
    featuresCol="features",
    labelCol="Is Highlight_numeric"
)

# Create a pipeline to assemble and transform the features and train the logistic regression model
pipeline = Pipeline(stages=indexers + encoders + [
    # medium_encoder, artist_encoder,
     assembler, logistic_regression])

# Fit the pipeline to the artwork DataFrame
model = pipeline.fit(artwork_df)

# Perform predictions on the artwork DataFrame
predictions = model.transform(artwork_df)

# Show the predicted labels and probabilities
predictions.select("Is Highlight_numeric", "prediction", "probability").show()


+--------------------+----------+--------------------+
|Is Highlight_numeric|prediction|         probability|
+--------------------+----------+--------------------+
|                   0|       0.0|[0.99999999158770...|
|                   1|       1.0|[7.07114400554553...|
|                   0|       0.0|[0.99999999242119...|
|                   0|       0.0|[0.99999999158770...|
+--------------------+----------+--------------------+

